<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/Sequential_Chats01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [ ]:
!wget https://raw.githubusercontent.com/JSJeong-me/GPT-Agent/main/autogen/requirements.txt
!pip install -r requirements.txt


In [ ]:
!pip install dask[dataframe]

In [8]:
from google.colab import userdata
import os
api_key = userdata.get('OPENAI_API_KEY')

In [9]:
llm_config={"model": "gpt-3.5-turbo",  "api_key": api_key } # os.environ.get("OPENAI_API_KEY")

In [10]:
from autogen import ConversableAgent

## Creating the needed agents

In [6]:
!curl -O https://raw.githubusercontent.com/JSJeong-me/GPT-Agent/main/autogen/utils.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   890  100   890    0     0   2119      0 --:--:-- --:--:-- --:--:--  2124


In [11]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-14 23:34:17] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-14 23:34:30] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 09-14 23:34:33] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [14]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [15]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message":
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [16]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Alice
customer_proxy_agent (to Onboarding Personal Information Agent):

Alice

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Great, thank you for sharing your name. Could you also tell me your location, please?

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: New York
customer_proxy_agent (to Onboarding Personal Information Agent):

New York

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

*******************************************************

## Print out the summary

In [17]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
    "name": "Alice",
    "location": "New York"
}


Alice from New York has shown interest in reading about cats.


Alice from New York has shown interest in reading about cats. Did you know that there is a town in Alaska called Talkeetna where a cat named Stubbs was the honorary mayor for 20 years?




## Print out the cost

In [18]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0001465, 'gpt-3.5-turbo-0125': {'cost': 0.0001465, 'prompt_tokens': 185, 'completion_tokens': 36, 'total_tokens': 221}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001465, 'gpt-3.5-turbo-0125': {'cost': 0.0001465, 'prompt_tokens': 185, 'completion_tokens': 36, 'total_tokens': 221}}}


{'usage_including_cached_inference': {'total_cost': 5.1500000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1500000000000005e-05, 'prompt_tokens': 67, 'completion_tokens': 12, 'total_tokens': 79}}, 'usage_excluding_cached_inference': {'total_cost': 5.1500000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1500000000000005e-05, 'prompt_tokens': 67, 'completion_tokens': 12, 'total_tokens': 79}}}


{'usage_including_cached_inference': {'total_cost': 0.00032800000000000006, 'gpt-3.5-turbo-0125': {'cost': 0.00032800000000000006, 'prompt_tokens': 272, 'completion_tokens': 128, 'total_tokens': 400}}, 'usage_excluding_cached_inference': {'total_cost